# MLflow KServe integration test
To properly run this notebook you need to deploy MLflow with Kubeflow according to [these](https://documentation.ubuntu.com/charmed-mlflow/en/latest/tutorial/mlflow-kubeflow/) instructions. After that you need to add three extra relations (for k-serve integration). 

```
juju relate kserve-controller:service-accounts resource-dispatcher:service-accounts
juju relate kserve-controller:secrets resource-dispatcher:secrets
juju relate kserve-controller mlflow-minio
``` 

Steps:
- start experiment 
- train model
- save artifact to MLflow
- create KServe inference service with the artifact
- make sure service is up
- run prediction
- remove the inference service

## Setup

In [ ]:
# Please check the requirements.in file for more details
!pip install -r requirements.txt

### Import required packages

In [ ]:
import mlflow
import numpy as np
import pandas as pd
import requests

from kubernetes import client as k8s_client, config as k8s_config
from mlflow.models.signature import infer_signature
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from tenacity import retry, stop_after_attempt, wait_exponential

## Download Data

In [ ]:
data = pd.read_csv(
    "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv",
    sep=";",
)
data.head()

In [ ]:
data.shape

## Preprocess Data

In [ ]:
TARGET_COLUMN = "quality"
train, test = train_test_split(data)

train_x = train.drop([TARGET_COLUMN], axis=1)
test_x = test.drop([TARGET_COLUMN], axis=1)
train_y = train[[TARGET_COLUMN]]
test_y = test[[TARGET_COLUMN]]

## Create MLflow experiment

In [ ]:
wine_experiment_name = "My Wine Experiment Kserve"
experiment = mlflow.get_experiment_by_name(wine_experiment_name)
experiment_id = (
    mlflow.create_experiment(name=wine_experiment_name)
    if experiment is None
    else experiment.experiment_id
)

In [ ]:
# check that the experiment was created successfully
assert (
    mlflow.get_experiment(experiment_id).name == wine_experiment_name
), f"Failed to create experiment {wine_experiment_name}!"

## Train and store model

In [ ]:
def experiment(alpha, l1_ratio):
    mlflow.sklearn.autolog()
    with mlflow.start_run(run_name="wine_models", experiment_id=experiment_id) as run:
        mlflow.set_tag("author", "kf-testing")
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        pred_y = lr.predict(test_x)
        mlflow.log_metric("rmse", np.sqrt(mean_squared_error(test_y, pred_y)))
        mlflow.log_metric("r2", r2_score(test_y, pred_y))
        mlflow.log_metric("mae", mean_absolute_error(test_y, pred_y))

        signature = infer_signature(test_x, pred_y)
        result = mlflow.sklearn.log_model(
            lr, "model", registered_model_name="wine-elasticnet", signature=signature
        )
        model_uri = f"{mlflow.get_artifact_uri()}/{result.artifact_path}"

    return run, model_uri

In [ ]:
run, model_uri = experiment(0.5, 0.5)

In [ ]:
model_uri

## Deploy Kserve's InferenceService

In [ ]:
from kubernetes.client import V1ObjectMeta
from kubernetes import client as k8s_client, config as k8s_config
from kserve import (
    constants,
    KServeClient,
    V1beta1InferenceService,
    V1beta1InferenceServiceSpec,
    V1beta1PredictorSpec,
    V1beta1SKLearnSpec,
)

In [ ]:
ISVC_NAME = "wine-regressor3"

isvc = V1beta1InferenceService(
    api_version=constants.KSERVE_V1BETA1,
    kind=constants.KSERVE_KIND_INFERENCESERVICE,
    metadata=V1ObjectMeta(
        name=ISVC_NAME,
        annotations={"sidecar.istio.io/inject": "false"},
    ),
    spec=V1beta1InferenceServiceSpec(
        predictor=V1beta1PredictorSpec(
            service_account_name="kserve-controller-s3",
            sklearn=V1beta1SKLearnSpec(storage_uri=model_uri),
        )
    ),
)

In [ ]:
client = KServeClient()
client.create(isvc)


@retry(
    wait=wait_exponential(multiplier=2, min=1, max=10),
    stop=stop_after_attempt(30),
    reraise=True,
)
def assert_isvc_created(client, isvc_name):
    """Wait for the Inference Service to be created successfully."""
    assert client.is_isvc_ready(ISVC_NAME), f"Failed to create Inference Service {isvc_name}."


assert_isvc_created(client, ISVC_NAME)

## Run Prediction

In [ ]:
isvc_resp = client.get(ISVC_NAME)
isvc_url = isvc_resp["status"]["address"]["url"]
print("Inference URL:", isvc_url)

In [ ]:
inference_input = {
    "instances": [[10.1, 0.37, 0.34, 2.4, 0.085, 5.0, 17.0, 0.99683, 3.17, 0.65, 10.6]]
}
response = requests.post(f"{isvc_url}/v1/models/{ISVC_NAME}:predict", json=inference_input)
print(response.text)

## Delete Inference Service

In [ ]:
client.delete(ISVC_NAME);

In [ ]:
@retry(
    wait=wait_exponential(multiplier=2, min=1, max=10),
    stop=stop_after_attempt(30),
    reraise=True,
)
def assert_isvc_deleted(client, isvc_name):
    """Wait for the Inference Service to be deleted."""
    try:
        # try fetching the ISVC to verify it was deleted successfully
        isvc = client.get(isvc_name)
        assert not isvc, f"Failed to delete Inference Service {isvc_name}!"
    except RuntimeError as err:
        assert "Not Found" in str(err), f"Caught unexpected exception: {err}"

In [ ]:
assert_isvc_deleted(client, ISVC_NAME)